In [1]:
import io
import progressbar
import os
import struct

In [2]:
class Constants:
    def __init__(self, max_bits = 18, ceil = 16):
        self.max_bits = max_bits
        self.ceil = ceil

In [8]:
class LZW:
    def __init__(self, max_bits = 16, ceil = 16):
        self.constants = Constants(max_bits, ceil)
        self.letters_counts={}
        self.file_str = ''
        self.file_path = ''
        self.codes = {}
        self.out_codes = {}
        self.output = ''
        self.decoded = ''
        self.max_length = 2**self.constants.max_bits
        self.main_length = 0
        self.ceil = self.constants.ceil
        self.main_codes = {}
        self.main_out_codes = {}
    
    
    def readFile(self, file_path):
        self.file_path = file_path
        self.file_str = io.open(file_path, mode="r", encoding="utf-8").read()
        self.file_str = self.file_str.replace('\n','\r\n')
        let_count = 1
        for let in self.file_str:
            if self.letters_counts.has_key(let):
                self.letters_counts[let] = self.letters_counts[let] + 1
            else:
                self.letters_counts[let] = 1
                self.codes[let] = len(self.codes)
        self.out_codes = {v: k for k, v in self.codes.iteritems()}
        self.main_codes = self.codes.copy()
        self.main_out_codes = self.out_codes.copy()
        self.main_length = len(self.codes)
                
            
    def encode(self):
        s = self.file_str[0]
        i = 1
        with progressbar.ProgressBar(max_value=len(self.file_str)) as bar:
            while i<len(self.file_str):
                c = self.file_str[i]
                if self.codes.get(s+c, None) != None:
                    s = s+c
                else:
                    bits = bin(self.codes[s])[2:]
                    self.output += bits.zfill(self.ceil)
                    self.codes[s+c] = len(self.codes)
                    s = c

                if len(self.codes) >= self.max_length:
                    self.codes = self.main_codes.copy()

                i += 1
                bar.update(i)
        bits = bin(self.codes[s])[2:]
        self.output += bits.zfill(self.ceil)
        
        
    def decode(self):
        self.ceil = self.constants.ceil
        s = None
        i = 0
        with progressbar.ProgressBar(max_value=len(self.output)) as bar:
            while i< len(self.output):
                k = int(self.output[i: i+self.ceil], 2)
                try:
                    entry = self.out_codes[k]
                except KeyError:
                    entry = s + s[0]

                self.decoded += entry

                if s != None:
                    self.out_codes[len(self.out_codes)] = s+entry[0]


                if len(self.out_codes) >= self.max_length:
                    self.out_codes = self.main_out_codes.copy()

                s = entry
                i += self.ceil
                bar.update(i)
                
    def printEncodedFile(self, name = 'encoded_file.bin'):
        newFile = open(name, "wb")
        newFile.write(struct.pack('{}H'.format(len(self.output)), *self.output))
        newFile.close()
        
    def printTryFile(self):
        str_to_print = ''
        i = 0
        while i < len(self.output):
            str_to_print += unichr(int(self.output[i: i+16], 2))
            i += 16
        file_to_print = io.open('omar.txt', mode="w", encoding="utf-8")
        file_to_print.write(str_to_print)
        file_to_print.close()
        print 'Done!'
            
            
    def calculateCompressionRatio(self):
        return float(os.path.getsize(self.file_path))/(len(self.output) / 8.0)

In [27]:
model = LZW()
model.readFile('omar.txt')
print model.file_str
#print sys.getsizeof(encoder.file_str)
#os.path.getsize(encoder.file_path)

عمر عمر عمر عمر عمر


In [28]:
model.encode()

100% (19 of 19) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [29]:
model.calculateCompressionRatio()

1.5454545454545454

In [30]:
model.decode()

100% (176 of 176) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


In [31]:
print model.decoded

عمر عمر عمر عمر عمر


In [32]:
errors = 0
if not len(model.file_str) == len(model.decoded):
    print 'length mismatch first len is {} and second is {}'.format(len(model.file_str),len(model.decoded))
else:
    for i in range(len(model.file_str)):
        if not model.file_str[i] == model.decoded[i]:
            errors +=1
errors

0